In [1]:
from __future__ import division
import itertools
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import pandas as pd
import math
from sklearn import metrics
from random import randint
from matplotlib import style
import seaborn as sns
from wrangle import clean_log_data
%matplotlib inline
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [2]:
df= clean_log_data()

In [3]:
df.index.dtype

dtype('<M8[ns]')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 900223 entries, 2018-01-26 09:55:03 to 2021-04-21 16:44:39
Data columns (total 11 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   date     900223 non-null  object
 1   time     900223 non-null  object
 2   path     900222 non-null  object
 3   user_id  900223 non-null  object
 4   ip       900223 non-null  object
 5   cohort   847330 non-null  object
 6   year     900223 non-null  object
 7   month    900223 non-null  object
 8   day      900223 non-null  object
 9   hour     900223 non-null  object
 10  weekday  900223 non-null  object
dtypes: object(11)
memory usage: 82.4+ MB


In [5]:
df.describe()

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-5-ea8415b8a3ee>", line 1, in <module>
    df.describe()
  File "/usr/local/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py", line 9940, in describe
    ldesc = [describe_1d(s) for _, s in data.items()]
  File "/usr/local/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py", line 9940, in <listcomp>
    ldesc = [describe_1d(s) for _, s in data.items()]
  File "/usr/local/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py", line 9923, in describe_1d
    return describe_categorical_1d(data)
  File "/usr/local/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py", line 9881, in describe_categorical_1d
    result = [data.count(), count_unique]
  File "/usr/local/anaconda3/lib/python3.7/site-packages/pandas/core/series.py", line 

KeyboardInterrupt: 

In [ ]:
df.cohort.isna().sum()

In [ ]:
#df.groupby(['ip']).count()

In [ ]:
df.groupby('weekday').count()

In [ ]:
df.groupby('weekday')['ip'].count()

In [ ]:
df.groupby(["weekday", "hour"])["ip"].count()

In [ ]:
df.groupby('weekday')["user_id"].sum()

In [ ]:
df.groupby('weekday')["ip"].sum()

Takeways:
- There are 900223 hits to the Codeup Curriculum 
- The data consists of a datetime object, the page that was being viewed, the user id, cohort id, and ip address.
- Students are assigned numbers (user_id) and grouped in to cohorts (cohort_id)
- There are 52893 nulls for cohort_id which could suggest instructor use of the website.
- Students often hit the website from different ip addresses, which suggests that students aren't always working from home.


#### Finding web dev vs data science students

In [ ]:
# Creating a column for datascience based on boolean mask. Looking for data science as True makes most sense as there are more web dev students than data science students.
df['is_data_science'] = (df.cohort.str.contains('Ada') | df.cohort.str.contains('Bayes') | df.cohort.str.contains('Curie') | df.cohort.str.contains('Darden') | df.cohort.str.contains('Easley')| df.cohort.str.contains('florence')) 
df.head()

#### Finding the amount of hits by hour

In [ ]:
df.groupby('hour')['ip'].count()

The most number of hits occurs at 0900. After 0900 the number of hits remains high until about 1500 then it tapers off and fluctuates until about 2300. After 0000, the number of hits decreases with the least number of hits being at 0300.

## Finding my data

In [ ]:
df.loc[df.ip=='173.174.172.52']

# Finding anomalies in data science students

In [ ]:
data_science= df.loc[df.is_data_science == True]

In [ ]:
data_science.head()

In [ ]:
data_science.groupby('hour')['ip'].count()

In [ ]:
sns.set(rc={'figure.figsize':(11.7,8.27)})
sns.boxplot(x=data_science.hour, y=data_science.weekday, hue=data_science.cohort, data=data_science)
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.xticks([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12 ,13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23])
plt.title("Distribution of Day of Week and Time Accessing Curriculum by Cohort")

In [ ]:
data_science.date.value_counts()

In [ ]:
# Lets use 70/30 train test split %

train_size = .70
n = data_science.shape[0]
test_start_index = round(train_size * n)
test_start_index

In [ ]:
# create train and test dataframe

train = data_science[:test_start_index] # everything up (not including) to the test_start_index
test = data_science[test_start_index:] # everything from the test_start_index to the end

In [ ]:
train.head()

In [ ]:
# lets plot train and test

plt.plot(train.index, train.ip)
plt.plot(test.index, test.ip)